In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="MachineLearning")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: MachineLearning
Azure region: eastus2
Subscription id: c6d23b07-48c4-47cb-803b-f5441d406570
Resource group: fast2ai


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
compute_target = "cluster-udacity"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=compute_target)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [10]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( {'--C': choice(0.1,2,10,100),
                               '--max_iter': choice(50,100,200)})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".", 
                compute_target=compute_target, 
                entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=10,
                                     max_concurrent_runs=4)

In [11]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [14]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
hyperdrive_model = best_run.register_model(model_name = 'BankModel', model_path = 'outputs/model.joblib')
print('best run :',best_run)

best run : Run(Experiment: udacity-project,
Id: HD_5a658ee1-8f3e-4570-b673-46d04750b8fc_0,
Type: azureml.scriptrun,
Status: Completed)


In [15]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [16]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
dataset = x.join(y)
train_data, test_data =  train_test_split(dataset, random_state = 123)

In [25]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=15,
    task="classification",
    enable_early_stopping = True,
    primary_metric="AUC_weighted",
    training_data=train_data,
    label_column_name="y",
    enable_onnx_compatible_models=True,
    n_cross_validations=3)

In [26]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output = False)
RunDetails(automl_run).show()

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [27]:
# Retrieve and save your best automl model.

from azureml.automl.runtime.onnx_convert import OnnxConverter
best_run_model, onnx_model= automl_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, file_path="./automl_model.onnx")